In [1]:
import math
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm

def generate_adjacency_matrix(N):
    # n_genes = 10
    a = np.random.randint(0, 3, (N, N))
    a[np.tril_indices(a.shape[0], -1)] = a.T[np.tril_indices(a.shape[0], -1)]
    b = np.fliplr(a)
    b[np.tril_indices(b.shape[0], -1)] = b.T[np.tril_indices(a.shape[0], -1)]

    b[b == 2] = -1
    np.fill_diagonal(b, 0)  # Remove self loops
    return b

def get_vector_that_provides_corr(rho, x1):
    n = x1.shape[0]  # length of vector, the number of samples for the same gene that we define in n_cols
    # ho = 0.6  # desired correlation = cos(angle)
    theta = math.acos(rho)  # corresponding angle
    x2 = np.random.normal(2, 0.5, n)  # new random data
    X = np.c_[x1, x2]  # matrix
    Xctr = pd.DataFrame(X).apply(lambda x: x - x.mean()).to_numpy()  # centered columns (mean 0)
    Id = np.diag(np.repeat(1, n))  # identity matrix
    Q = np.linalg.qr(Xctr[:, 0].reshape(n, 1))[0].flatten()  # QR-decomposition, just matrix Q
    P = np.outer(Q, Q)  # projection onto space defined by x1
    x2o = np.dot((Id - P), Xctr[:, 1])  # x2ctr made orthogonal to x1ctr
    Xc2 = np.c_[Xctr[:, 0], x2o]  # bind to matrix
    Y = np.dot(Xc2, np.diag(1 / np.sqrt(np.sum(np.power(Xc2, 2), axis=0))))  # scale columns to length 1
    x = Y[:, 1] + (1 / np.tan(theta)) * Y[:, 0]
    x_norm = (x - x.mean()) / (x.std())
    return x_norm


def simulate_expression_from_adjacency(adjamat, n_samples=100, simdata_noise=0.1):
    # noise = 0.5 # .1 strong correlation, 3 weak correlation
    # random initialize data matrix
    # n_samples = 40 # number of samples: 40 so we later split into 20 cases and 20 controls
    n_rows = adjamat.shape[0]  # number of genes
    mu, sigma = 1, 1  # mean and standard deviation
    dataset = np.random.normal(mu, sigma, n_samples * n_rows).reshape(n_rows, n_samples)
    already_modified = np.repeat(0, n_rows)  # N=n1+n2 genes
    already_modified[0] = 1  # leave the first gene alone, base case

    for i in range(adjamat.shape[1]):
        for j in range(i + 1, adjamat.shape[1]):
            # print(f'Considering row: {i}, column: {j} of A')
            if (adjamat[i, j] == 1) & (already_modified[j] == 0):
                # print(i,j)
                # print(A[i,j])
                dataset[j, :] = dataset[i, :] +  np.random.normal(0, simdata_noise, dataset.shape[1])
                already_modified[j] = 1
            elif (adjamat[i, j] == -1) & (already_modified[j] == 0):
                # print(i,j)
                # print(A[i,j])
                dataset[j, :] = -dataset[i, :] + np.random.normal(0, simdata_noise, dataset.shape[1])
                already_modified[j] = 1
            elif (already_modified[j] == 1) & (already_modified[i] == 0):
                #if j is  modified, we accordingly modify i. If i has been modified, we do nothing.
                dataset[i,:] = dataset[j,:] + np.random.normal(0, simdata_noise, dataset.shape[1])
                already_modified[i] = 1
    ds = pd.DataFrame(dataset)
    ds.columns = 'sample_' + ds.columns.astype(str)
    ds.index = 'gene_' + ds.index.astype(str)
    return ds

def ground_truth_from_adjamat(adjamat):
    genenames = ['gene_' + s for s in list(str(i) for i in range(n_genes))]

    gene_A = []
    gene_B = []
    rels = []
    for i in range(adjamat.shape[1]):
        for j in range(i + 1, adjamat.shape[1]):
            if adjamat[i][j] != 0:
                #print(i, j, adjamat[i, j])
                gene_A.append(i)
                gene_B.append(j)
                rels.append(np.where(adjamat[i, j] > 0, "[+]", "[-]"))

    pairs = pd.DataFrame({'gene_A': gene_A, 'gene_B': gene_B, 'correlation': rels})
    mygenedict = dict(zip(range(adjamat.shape[1]), genenames))
    pairs.gene_A = pairs.gene_A.map(mygenedict)
    pairs.gene_B = pairs.gene_B.map(mygenedict)
    return pairs

Write random matrix of dimensions n_rows (genes) x n_samples (columns)

In [ ]:
n_samples = 250
n_rows = 5000
mu, sigma = 1, 1  # mean and standard deviation

print('Writing random distribution of genes')

with open ('dataset_'+ str(n_rows) + '_' + str(n_samples) + '.csv', 'a+') as f:
    sColumns = "gene,"
    for i in range(n_samples):
        sColumns = sColumns+"sample_"+str(i)
        if(i < n_samples-1):
            sColumns = sColumns + ","
    sColumns = sColumns + '\n'
    f.write(sColumns)
    for row in tqdm(range(n_rows)):
        vector = np.random.normal(mu, sigma, n_samples * 1)
        f.write(str(row)+","+",".join(str(x) for x in vector)+'\n')
    f.close()